In [2]:
import boto3
import json
import pandas as pd
from pandas import json_normalize
from datetime import date
from dateutil.relativedelta import *

In [3]:
COUNTRY_FOLDERS = ["DP00011-US",
                 "DP00286-DE",
                 "DP00300-FR",
                 "DP00423-CA",
                 "DP00483-GB",
                 "DP00514-ES",
                 "DP00544-IT",
                 "DP00554-AU",
                 "DP00726-TR",
                 "DP01101-BR",
                 "DP01102-NL",
                 "DP01103-SE",
                 "DP01104-PL",
                 "DP01107-SA",
                 "DP01110-SG"]
TODAY = date.today()

In [4]:
start_date = (TODAY - relativedelta(months=1)).replace(day=1)
year = start_date.year
month = start_date.month
if month < 10:
    month = f"0{month}"

start_date_str = start_date.strftime("%Y-%m-%d")
end_date_str = (start_date + relativedelta(day=1, months=+1, days=-1)).strftime("%Y-%m-%d")

In [5]:
s3 = boto3.resource('s3')

netppm = pd.DataFrame()
for country in COUNTRY_FOLDERS:
    content_object = s3.Object('s3-euc1-eadf-datalayer-raw-d-828952267417',
                               f"DPG0003-Amazon/{country}/VC-API/Reports/Vendor-Net-Pure-Product-Margin/Month/{year}/{month}/01/{start_date_str}_{end_date_str}.json")
    file_content = content_object.get()['Body'].read().decode('utf-8')
    json_content = json.loads(file_content)
    df = json_normalize(json_content["netPureProductMarginByAsin"])
    df["Country"] = country[-2:]
    if netppm.empty:
        netppm = df
    else:
        netppm = netppm.append(df)

In [9]:
netppm

,startDate,endDate,asin,netPureProductMargin,Country
0,2023-01-01,2023-01-31,B008H1FKFE,0.6768,US
1,2023-01-01,2023-01-31,B008MV944M,0.5452,US
2,2023-01-01,2023-01-31,B010KCOTCW,0.7790,US
3,2023-01-01,2023-01-31,B010KCOVNO,0.6191,US
4,2023-01-01,2023-01-31,B01D6KCY4K,0.2795,US
...,...,...,...,...,...
105,2023-01-01,2023-01-31,B0B2RFZVWJ,0.3086,SG
106,2023-01-01,2023-01-31,B0BC156911,0.1657,SG
107,2023-01-01,2023-01-31,B0BC179WV1,0.3324,SG
108,2023-01-01,2023-01-31,B0BC17C3NQ,0.1095,SG
